In [169]:
import scipy
import matplotlib.pyplot as plt
import numpy as np

In [168]:
N = [0,1]
E = [1,0]
S = [0,-1]
W = [-1,0]

def inters_rv():
    return round(scipy.stats.gamma.rvs(a=2, loc=0, scale=1) + 2.0)

class Node:
    def __init__(self, x, y, orig_dist=0, inters_dist=0, neigh=[], parent=None):
        self.x = x
        self.y = y
        self.orig_dist = orig_dist
        self.inters_dist = inters_dist
        self.neigh = neigh
        self.parent = parent

    def non_neigh(self):
        nn = [N, E, S, W]
        for n in self.neigh:
            nn.remove([n.x - self.x, n.y - self.y])

        if self.parent!=None:
            nn.remove([self.parent.x - self.x, self.parent.y - self.y])
        
        return nn

    def coord_str(self):
        return('(%d,%d)' % (self.x, self.y))

    def __str__(self):
        return '(%d,%d,n=%d,od=%d,id=%d) <= %s' % (self.x, self.y, len(self.neigh), self.orig_dist, self.inters_dist, self.parent)

def draw_map(origin, existing, new):
    for y in range(-5,6):
        for x in range(-5,6):
            if origin.x==x and origin.y==y:
                print('o', end='')
            if new.x==x and new.y==y:
                print('+', end='')
            if '(%d,%d)' % (x,y) in existing:
                print('▪', end='')
            else:
                print(' ', end='')
        print('')

o = Node(0,0)
origins = [
    o
]
existing = {
    o.coord_str(): o
}

wiggle = 3
for i in range(0,10):
    print("\nIteration %d" % i)
    min_od = 0
    od = 0

    candidates = []
    search = origins
    while od<min_od+wiggle-1:
        adj = []
        od += 1
        for n in search:
            print('Visiting %s' % n)
            for a in n.neigh:
                print('New search: %s' % a)
                adj.append(a)
                existing[a.coord_str()] = a

            if False:
                    
                eligible = n.non_neigh()
            for c in n.non_neigh():
                if '(%d,%d)' % (c[0], c[1]) in existing:
                    continue
                if min_od==0:
                    min_od = n.orig_dist+1
                cand = Node(
                    n.x+c[0],
                    n.y+c[1],
                    orig_dist=n.orig_dist+1,
                    inters_dist=n.inters_dist+1,
                    parent=n,
                    neigh=[],
                )
                print('Found canditate: %s' % cand)
                candidates.append(cand)
            
        search = adj

    if len(candidates)==0:
        print('No candidates found')
        break 
    new = np.random.choice(candidates)
    print('Adding node: %s' % new)
    new.parent.neigh.append(new)
    existing[new.coord_str()] = new

    draw_map(o, existing, new)



Iteration 0
Visiting (0,0,n=0,od=0,id=0) <= None
Found canditate: (0,1,n=0,od=1,id=1) <= (0,0,n=0,od=0,id=0) <= None
Found canditate: (1,0,n=0,od=1,id=1) <= (0,0,n=0,od=0,id=0) <= None
Found canditate: (0,-1,n=0,od=1,id=1) <= (0,0,n=0,od=0,id=0) <= None
Found canditate: (-1,0,n=0,od=1,id=1) <= (0,0,n=0,od=0,id=0) <= None
Adding node: (0,1,n=0,od=1,id=1) <= (0,0,n=0,od=0,id=0) <= None
           
           
           
           
           
     o▪     
     +▪     
           
           
           
           

Iteration 1
Visiting (0,0,n=1,od=0,id=0) <= None
New search: (0,1,n=0,od=1,id=1) <= (0,0,n=1,od=0,id=0) <= None
Found canditate: (1,0,n=0,od=1,id=1) <= (0,0,n=1,od=0,id=0) <= None
Found canditate: (0,-1,n=0,od=1,id=1) <= (0,0,n=1,od=0,id=0) <= None
Found canditate: (-1,0,n=0,od=1,id=1) <= (0,0,n=1,od=0,id=0) <= None
Visiting (0,1,n=0,od=1,id=1) <= (0,0,n=1,od=0,id=0) <= None
Found canditate: (1,1,n=0,od=2,id=2) <= (0,1,n=0,od=1,id=1) <= (0,0,n=1,od=0,id=0) <= None
Found ca